<h1><strong>CSCI 4455/5455&ndash; Fall 2021</strong></h1>

<h2><strong>Assignment 2 - Data Warehousing & OLAP</strong></h2>

<h3><strong><span style="color:#cc3300;">Due: October 27, 11:59pm </span></strong></h3>

<h3><strong>Your name:</strong></h3>

<ul>
<li style="text-align: justify;">Please note that you must do this assignment&nbsp;<span style="color: #cc3300;"><strong><u>individually</u></strong></span>. Using automatic tools, your code will be checked against other submissions and other existing resources (such as websites and books).</li>
<li style="text-align: justify;">This assignment is more extensive and might take longer than previous assignments to finish. <span style="color: #339966;"><strong><u>Please start early on.</u></strong></span></li>
<li style="text-align: justify;">Review the lecture notes before starting with this assignment. Then, thoroughly read this document before starting with the implementation or thinking about the solution.</li>
<li style="text-align: justify;">If you have technical questions about Python, please Google the error messages and share the error message alongside the solution that got it fixed on Microsoft Teams, as your classmates may run into the same issues.</li>
<li style="text-align: justify;">Check Canvas regularly for possible clarifications and updates.</li>
<li style="text-align: justify;">In this assignment, you will design and implement a data warehouse from scratch and use the data warehouse to answer some analytical decision-making questions.</li>
</ul>


<h2>Dataset</h2>

<p style="text-align: justify;">The dataset for this assignment can be found on this <a href="https://www.kaggle.com/rounakbanik/the-movies-dataset">Kaggle link.</a>
Please read the data described on this page. We have also uploaded a copy of the data archive in Canvas for your reference. <br>
    
After you download and extract the archive, you should be able to find the following CSV files:
</p>

<img src="Files.png">

<h2>Implementation</h2>

<p style="text-align: justify;">Consider yourself as a data scientist employee of a tech company. One day your boss gives you the above dataset and asks you to design and implement a data warehouse model for the company. Your data warehouse should enable efficient answering of the following queries:<br>
    
<ol>
<li><strong>Details of the Production Company:</strong></li>
</ol>
<ul>
<li style="list-style-type: none;">
<ul style="list-style-type: square;">
<li>Revenue per year (rpy.csv)</li>
<li>Budget per year (bpy.csv)</li>
<li>Profit per year (ppy.csv)</li>
<li>Average popularity of the produced movies per year (apy.csv)</li>
<li>Releases by genre per year (rgpy.csv)</li>
</ul>
</li>
</ul>
<ol start="2">
<li><strong>Movie Genre Details:</strong></li>
</ol>
<ul>
<li style="list-style-type: none;">
<ul style="list-style-type: square;">
<li>Revenue by genre per year (regpy.csv)</li>
<li>Budget by genre per year (bgpy.csv)</li>
<li>Profit by genre per year (pgpy.csv)</li>
<li>Most popular genre by year (mpgy.csv)</li>
</ul>
</li>
</ul>
</p>

<h2>Question 1</h2>

<p style="text-align: justify;">Prepare and submit your design of the data warehouse schema. Choose a schema suitable for your database based on the discussion about data warehouse schemas in class. (our suggestion is using the star schema, but you can choose other schemas as well)

Note: You can insert your answer as a picture. if that's the case, make sure to submit a single zip file consisting this notebook and the picture file together.
</p>

<p><span style="color: #ff0000;"><strong>-- Write your answer here --</strong></span></p>

<h2>Question 2</h2>

<p style="text-align: justify;">Implement a database to store your data warehouse based on your design.</p>

<p style="text-align: justify;">Your code should take the movies_metadata.csv path from the user to populate the database</p>

<p style="text-align: justify;"><strong>Hint1:</strong> You will need to design a database for this part of the assignment. You can choose between Pandas Dataframe or <a href="https://realpython.com/python-mysql/">MySQL</a> database to implement your database. If you&rsquo;re not sure about which one to choose, please take a look at this <a href="https://towardsdatascience.com/sql-vs-pandas-which-one-to-choose-in-2020-a98f236829ef">link</a>.</p>

<p style="text-align: justify;"><strong>Hint 2:</strong> Some of the items, like the genre column, have been stored as a list of dictionaries. You can use the <a href="https://www.geeksforgeeks.org/read-json-file-using-python/">“json.loads()”</a> and <a href="https://www.kite.com/python/docs/ast.literal_eval">“ast.literal_eval()”</a>  functions from json and ast libraries, respectively, to parse through these types of columns quickly.

In [1]:
# wrtie the code you implemented to create and populate the database here
# you can insert more cells below, if needed
import pandas as pd
import numpy as np
import ast
#please upload data set to your Notebook jupyter
df_movies_metadata = pd.read_csv(r'movies_metadata.csv')
df_credits = pd.read_csv(r'credits.csv')
df_ratings = pd.read_csv(r'ratings_small.csv')
df_links = pd.read_csv(r'links_small.csv')
df_keywords = pd.read_csv(r'keywords.csv')
#preprocesssing movies metadata
df_movies_metadata['revenue'] = df_movies_metadata['revenue'].replace(0,np.nan)
df_movies_metadata['budget'] = pd.to_numeric(df_movies_metadata['budget'], errors='coerce')
df_movies_metadata['budget'] = df_movies_metadata['budget'].replace(0, np.nan)
df_movies_metadata[df_movies_metadata['budget'].isnull()].shape
#create profit feature
df_movies_metadata['profit'] = df_movies_metadata['revenue'] - df_movies_metadata['budget']
#create year feature
df_movies_metadata['year'] = pd.to_datetime(df_movies_metadata['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)
#extract production companies json dataset and create companies name feature
df_movies_metadata['production_companies'] = df_movies_metadata['production_companies'].fillna('[]').apply(ast.literal_eval)
df_movies_metadata['production_companies'] = df_movies_metadata['production_companies'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
s = df_movies_metadata.apply(lambda x: pd.Series(x['production_companies']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'companies'
com_df_movies_metadata = df_movies_metadata.drop('production_companies', axis=1).join(s)


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
<ipython-input-1-a1f4d22d9e42>:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = df_movies_metadata.apply(lambda x: pd.Series(x['production_companies']),axis=1).stack().reset_index(level=1, drop=True)


In [2]:
def clean_numeric(x):
    try:
        return float(x)
    except:
        return np.nan
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [3]:
#preprocessing
com_df_movies_metadata['popularity'] = com_df_movies_metadata['popularity'].apply(clean_numeric).astype('float')
com_df_movies_metadata['vote_count'] = com_df_movies_metadata['vote_count'].apply(clean_numeric).astype('float')
com_df_movies_metadata['vote_average'] = com_df_movies_metadata['vote_average'].apply(clean_numeric).astype('float')
com_df_movies_metadata['vote_average'] = com_df_movies_metadata['vote_average'].replace(0, np.nan)
#extraction genres feature(json)
com_df_movies_metadata['genres'] = com_df_movies_metadata['genres'].fillna('[]').apply(ast.literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
s = com_df_movies_metadata.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
df_full = com_df_movies_metadata.drop('genres', axis=1).join(s)
#preprocessing for left create data warehouse
df_full['id'] = df_full['id'].apply(convert_int)
df_full = df_full.drop([19730, 29503, 35587])
df_full['id'] = df_full['id'].astype('int')
#join movies metadata and credits
df_full = df_full.merge(df_credits, on='id')
#extract json dataset
df_full['cast'] = df_full['cast'].apply(ast.literal_eval)
df_full['crew'] = df_full['crew'].apply(ast.literal_eval)
df_full['cast'] = df_full['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
#join movies meta data and keywords
df_full = df_full.merge(df_keywords,on='id')
#join movies meta data and links
df_full = df_full.merge(df_links,how='left',left_on='id',right_on='movieId')
df_full.drop('movieId',axis=1,inplace=True)
#join movie meta data and ratings
df_full = df_full.merge(df_ratings,how='left',left_on='id',right_on='movieId')
df_full.drop('movieId',axis=1,inplace=True)

<ipython-input-3-328223645f50>:8: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = com_df_movies_metadata.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)


<h2>Question 3</h2>

<p style="text-align: justify;">Write and implement queries to answer all queries mentioned above under the "Implementation" Section on Details of the Production Company and Move Genre Details. Your function should write the results of the aforementioned queries in separate CSV files using the provided file names, respectively. Here are some samples of outputs (numbers and names are hypothetical).

<img src="output_sample_b.png" width="600" 
     height="400">
    
<img src="output_sample_r.png" width="600" 
     height="400">
    
<img src="output_sample_p.png" width="600" 
     height="400">

<p style="text-align: justify;">Make sure that you are using your designed database in Question 1 to answer these queries.</p>


<p style="text-align: justify;"><strong>Hint 3</strong>: Most of the data you need to answer the queries are in the “movies_metadata.csv” file. </p>


In [4]:
# wrtie the code that you implemented to answer the queries here
# you can insert more cells below, if needed
#Revenue per year (rpy.csv)
df_full.groupby(['companies','year']).agg({'revenue':'sum'}).to_csv('rpy.csv')
#Budget per year (bpy.csv)
df_full.groupby(['companies','year']).agg({'budget':'sum'}).to_csv('bpy.csv')
#Profit per year (ppy.csv)
df_full.groupby(['companies','year']).agg({'profit':'sum'}).to_csv('ppy.csv')
#Average popularity of the produced movies per year (apy.csv)
df_full.groupby(['companies','year']).agg({'popularity':'mean'}).to_csv('apy.csv')
#Releases by genre per year (rgpy.csv)
df_full.groupby(['companies','genre','year']).agg({'release_date':'count'}).to_csv('rgpy.csv')
#Revenue by genre per year (regpy.csv)
df_full.groupby(['genre','year']).agg({'revenue':'sum'}).to_csv('regpy.csv')
#Budget by genre per year (bgpy.csv)
df_full.groupby(['genre','year']).agg({'budget':'sum'}).to_csv('bgpy.csv')
#Profit by genre per year (pgpy.csv)
df_full.groupby(['genre','year']).agg({'profit':'sum'}).to_csv('pgpy.csv')
#Most popular genre by year (mpgy.csv)
df_full.groupby(['year','genre']).agg({'popularity':'max'}).to_csv('mgpy.csv')

<h2>Question 4</h2>

Implement the following two queries:

<p><strong>a)</strong> Implement a function that takes a movie&rsquo;s IMDB id as an input and returns the following dictionary:</p>
<ul>
<li>Key: The number of users who voted for the movie</li>
<li>Value: The average rating of the movie by those users</li>
</ul>

<p style="text-align: justify;">Make sure that you are using your designed database in Question 1 here.</p>

<p style="text-align: justify;"><strong>Hint:</strong> To write the rating function, you need to work on links_small, rating_small files with the movies_metadata.csv to connect these tables.</p>

In [5]:
# wrtie the code that you implemented to answer the query here
# you can insert more cells below, if needed
def find_count_mean_rating(imdb_id):
    count_rate = df_full.loc[df_full.imdb_id==imdb_id].rating.count()
    mean_rate = df_full.loc[df_full.imdb_id==imdb_id].rating.mean()
    return {count_rate:mean_rate}

<p style="text-align: justify;"><strong>b)</strong> Implement a function that takes a list of movie IMDB ids (up to 20 movies at a time) and returns a CSV file with corresponding records in the following format: the movie id, movie name, number of users who rated that movie, and the average rating on each row. Your code should <em>efficiently</em> generate this report using the ratings.csv and links.csv files.</p>

In [6]:
# wrtie the code that you implemented to answer the query here
# you can insert more cells below, if needed
def create_dataset_mean_count(imdb_id_list):
    df = pd.DataFrame(columns=['movieId','movieName','number_of_rate','mean_of_rate'])
    for imdb_id in imdb_id_list:
        count_rate = df_full.loc[df_full.imdb_id==imdb_id].rating.count()
        mean_rate = df_full.loc[df_full.imdb_id==imdb_id].rating.mean()
        movie_name = df_full.loc[df_full.imdb_id==imdb_id].title.unique()[0]
        movie_id = df_full.loc[df_full.imdb_id==imdb_id].id.unique()[0]
        df = df.append({'movieId':movie_id,'movieName':movie_name,'number_of_rate':count_rate,'mean_of_rate':mean_rate},ignore_index=True)
    df.to_csv('report.csv')

<h3>Coding Considerations</h3>
<p style="text-align: justify;">You must consider the following in your implementations:</p>
<ul>
<li class="a"><span> You are allowed to use your desired Python libraries in this assignment. </span>
<li class="a"><span> The output CSV files should be generated in the same directory of your code when you run it. Use the naming provided in the Implementation section. </span>
<li class="a"><span> You should write your code in this Jupyter Notebook (*.ipynb) file which can generate the required reports itself.</span>
<li class="a"><span> Your code should be self-explanatory. Make sure you add comments and your output files are formatted correctly. You might lose up to 30 points for bad code quality (readability, modularity, comments, efficiency, etc.) and formatting of the output files.</span></li>
</ul>

<h2>Submission</h2>
<p style="text-align: justify;">Your python file or Jupyter Notebook file must generate all the abovementioned reports when you run it. If you use any specific library to run your code, add documentation.</p>
<p style="text-align: justify;">You need to submit a Jupyter Notebook (*.ipynb) file of your code, meeting the above requirements, in Canvas. <span style="background-color: #ffff99;">The file name should be in <strong>FirstName_LastName</strong> format</span>.</p>
<p style="text-align: justify;"><span style="background-color: #ffff99;">DO NOT INCLUDE EXTRA FILES, SUCH AS THE INPUT DATASETS</span>, in your submission; Unless you are sending a picture for Question 1.</p>
<p style="text-align: justify;">Please download your assignment after submission and make sure it is not corrupted or empty! We will not be responsible for corrupted submissions and will not take a resubmission after the deadline.</p>

<h2>Need Help?</h2>
<p>If you need help with this assignment, please get in touch with Erfan (on MS Teams or via email at <a href="mailto:erfan.jafarikhademzavareh@ucdenver.edu">Erfan.jafarikhademzavareh@ucdenver.edu</a>) or go to his office hours.</p>
<p>&nbsp;</p>
<p>You are highly encouraged to ask your question on the designated channel for Assignment 2 on Microsoft Teams (not necessarily monitored by the instructor/TA). Feel free to help other students with general questions. However, DO NOT share your solution.</p>